<a href="https://colab.research.google.com/github/Charl35X/CSC421_Mini-Project/blob/main/CSC411_Mini_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import random
import string
import xml.etree.ElementTree as ET
import threading
import queue
import time
import os

class ITStudent:
    """Class to generate random student data"""
    def __init__(self):
        self.name = ''.join(random.choices(string.ascii_uppercase + string.ascii_lowercase, k=10))
        self.id = ''.join(random.choices(string.digits, k=8))
        self.programme = 'IT'
        self.courses = {f'course{i}': random.randint(0, 100) for i in range(1, 6)}

class Producer(threading.Thread):
    """Thread to produce student data, convert it to XML, and add it to a shared buffer"""
    def __init__(self, buffer, empty_semaphore, full_semaphore):
        threading.Thread.__init__(self)
        self.buffer = buffer
        self.empty_semaphore = empty_semaphore
        self.full_semaphore = full_semaphore

    def run(self):
        for i in range(1, 11):
            try:
                student = ITStudent()
                # Convert student data to XML and save it to a file
                root = ET.Element("ITStudent")
                ET.SubElement(root, "name").text = student.name
                ET.SubElement(root, "id").text = student.id
                ET.SubElement(root, "programme").text = student.programme
                courses = ET.SubElement(root, "courses")
                for course, mark in student.courses.items():
                    course_element = ET.SubElement(courses, "course")
                    ET.SubElement(course_element, "name").text = course
                    ET.SubElement(course_element, "mark").text = str(mark)
                tree = ET.ElementTree(root)
                tree.write(f"student{i}.xml")
                # Add the file number to the buffer
                self.empty_semaphore.acquire()  # Wait if buffer is full
                self.buffer.put(i)
                self.full_semaphore.release()  # Signal that buffer is not empty
                time.sleep(random.randint(1, 5))
            except Exception as e:
                print(f"An error occurred in the producer: {e}")
